# Introduction to Naive Bayes
*Interactive [article](https://gormanalysis.com/introduction-to-naive-bayes/)*

$$ p(C_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid C_k) p(C_k)}{p(\mathbf{x})}$$

I think there’s a rule somewhere that says “You can’t call yourself a data scientist until you’ve used a Naive Bayes classifier”.  It’s extremely useful, yet beautifully simplistic.  This article is my attempt at laying the groundwork for Naive Bayes in a practical and intuitive fashion.

Let’s start with a problem to motivate our formulation of Naive Bayes.

Suppose we own a professional networking site similar to LinkedIn. Users sign up, type some information about themselves, and then roam the network looking for jobs/connections/etc. Until recently, we only required users to enter their current job title, but now we’re asking them what industry they work in. New users are supplying this info as they sign up, but old users aren’t bothering to update their information. So, we need to build a text classification model to do it for them. Our data looks like this

In [1]:
import pandas as pd

job_titles = pd.read_csv("_Data/jobtitles.csv")
job_titles.head()

,job_title,job_category
0,underwriter manager,finance
1,mortgage data analyst,finance
2,junior underwriter,finance
3,sales manager,sales
4,junior medical sales associate,sales


In [2]:
job_titles.tail()

,job_title,job_category
7,senior data analyst,technology
8,data analyst,technology
9,data manager,technology
10,data analyst manager,NaN
11,junior data analyst,NaN


Our goal – to estimate the probability those two unlabeled job titles should be categorized as Technology, Sales, or Finance, at which point we can make our best guess for the user. Formalizing this a bit, we want to find $p(C_k \mid \text{job_title})$ where $C_1,C_2$ and $C_3$ are the classes Technology, Sales and Finance. (Note: this type of problem is called [Document Classification](https://en.wikipedia.org/wiki/Document_classification).) 

How about that first unlabeled title, “data analyst manager”?  We should probably label it as Technology, but how do we train a computer to figure that out?  If we had trillions of training samples we might be able to estimate $p(C_k \mid \text{job_title}=\text{"data analyst manager"})$ empirically (i.e. by measuring the relative frequency of each class for samples where `job_title="data analyst manager"`). Unfortunately we only have 10 training samples (none of which have the title “data analyst manager”) so we’ll have to be a little more creative in our approach.

The word “data” seems pretty important.  It occurs in all of the Technology samples, none of the Sales samples and only one of the Finance samples.  On the other hand the word “manager” appears in every single category, so it’s probably not as useful.  The big takeaway here is that we can use word occurrences to build a probabilistic model.  Let’s start tracking words then...

In [3]:
words = job_titles['job_title'].str.get_dummies(' ')
job_titles = pd.concat([job_titles, words], axis=1)

### Finance

In [4]:
job_titles[job_titles['job_category'] == 'finance'].drop(['job_category'], axis=1).head()

,job_title,analyst,associate,data,junior,manager,medical,mortgage,sales,senior,underwriter
0,underwriter manager,0,0,0,0,1,0,0,0,0,1
1,mortgage data analyst,1,0,1,0,0,0,1,0,0,0
2,junior underwriter,0,0,0,1,0,0,0,0,0,1


### Sales

In [5]:
job_titles[job_titles['job_category'] == 'sales'].drop(['job_category'], axis=1).head()

,job_title,analyst,associate,data,junior,manager,medical,mortgage,sales,senior,underwriter
3,sales manager,0,0,0,0,1,0,0,1,0,0
4,junior medical sales associate,0,1,0,1,0,1,0,1,0,0
5,senior sales manager,0,0,0,0,1,0,0,1,1,0
6,sales manager,0,0,0,0,1,0,0,1,0,0


### Technology

In [6]:
job_titles[job_titles['job_category'] == 'technology'].drop(['job_category'], axis=1).head()

,job_title,analyst,associate,data,junior,manager,medical,mortgage,sales,senior,underwriter
7,senior data analyst,1,0,1,0,0,0,0,0,1,0
8,data analyst,1,0,1,0,0,0,0,0,0,0
9,data manager,0,0,1,0,1,0,0,0,0,0


### Unlabeled

In [7]:
job_titles[job_titles['job_category'].isnull()].drop(['job_category'], axis=1).head()

,job_title,analyst,associate,data,junior,manager,medical,mortgage,sales,senior,underwriter
10,data analyst manager,1,0,1,0,1,0,0,0,0,0
11,junior data analyst,1,0,1,1,0,0,0,0,0,0


Updating our model a bit, we want to find $p(C_k \mid \mathbf{x})$ where $\mathbf{x}$ is our feature vector of word occurences. In the case of "data analyst manager" $$\mathbf{x} = (x_1, x_2, ...,x_{10}) = (1,0,1,0,1,0,0,0,0,0)$$

--- 


Before we continue, let’s review some probability rules and [Bayes’ Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem)

$ P(A \mid B) = \frac{P(A \cap B)}{P(B)},\text{ if }P(B) \neq 0 $

$ P(B \mid A) = \frac{P(A \cap B)}{P(A)},\text{ if }P(A) \neq 0 $

$ \Rightarrow P(A \cap B) = P(A \mid B) P(B) = P(B \mid A) P(A),$

$ \Rightarrow P(A \mid B) = \frac{P(B \mid A) P(A)}{P(B)},\text{ if }P(B) \neq 0.$

For us this means

$p(C_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid C_k) p(C_k)}{p(\mathbf{x})}$

Let's break down those pieces:

- $p(C_k)$ - frequency of class $C_k$ / total number of samples
- $p(\mathbf{x} \mid C_k)$ - frequency of $\mathbf{x}$ / number of samples, where class = $C_k$ 
- $p(\mathbf{x})$ = frequency of $\mathbf{x}$ / total number of samples

---

### $p(C_k)$

This is the easiest part. To calculate $p(C_k)$ we can use empirical relative frequencies given by our training data.

In [8]:
mask = ~job_titles['job_category'].isnull()
categories = job_titles[mask]['job_category'].unique()

n = mask.sum()
p_cat = list([0.0]*len(categories))
for i, c in enumerate(categories):
    m = (job_titles[mask]['job_category'] == c).sum()
    p_cat[i] = m*1.0/n
    print("p(%s)= %i/%i = %.5f" % (c, m, n, p_cat[i]))

p(finance)= 3/10 = 0.30000
p(sales)= 4/10 = 0.40000
p(technology)= 3/10 = 0.30000


These probabilities are called “priors”. Our method of estimating them using the training data is common, but not necessary. Suppose we have reason to believe that the true priors for Technology, Sales, and Finance are 0.2, 0.4, and 0.4 respectively. With a Naive Bayes model, we can just plug those babies in for $p(C_k)$ and our model will adjust accordingly. By contrast, using priors for tree based models like Random Forest is not nearly as easy.

--- 

### $p(\mathbf{x} \mid C_k)$

Now let’s consider $p(\mathbf{x} \mid C_k)$. Using our training data we would calculate $p(\mathbf{x} \mid \text{Technology})$ to be 0. (Remember $p(\mathbf{x} \mid \text{Technology})$ represents the probability that only and all of the words “data”, “analyst”, and “manager” appear in a random job title given that it’s in the Technology class. In our training samples this never occurred, so our empirical probability estimate is 0.) This is a problem. We know $p(\mathbf{x} \mid \text{Technology})$ should be greater than 0, but we don’t have enough (or in this case, any) samples to accurrately estimate it. The way we’ll get around this problem is to make a naive assumption – we’ll assume that the features (i.e. the occurrence of words in a job title) are independent variables. Obviously this is not true. When the word “data” appears in a job title, it immediately increases the probability that the word “analyst” appears in the title. However let’s assume the assumption is valid (or close to being valid). Then

$$p(\mathbf{x} \mid C_k) = p(x_1 \mid C_k)p(x_2 \mid C_k) ... p(x_{10} \mid C_k)$$

For our example “data analyst manager” this means

$p(\mathbf{x} \mid \text{Technology}) = p(x_1 = 1 \mid Tech) \cdot p(x_2 = 0 \mid Tech)\cdot  ... \cdot p(x_{10} = 0 \mid Tech)$

Similarly, we can calculate $p(\mathbf{x} \mid \text{Sales})$ and $p(\mathbf{x} \mid \text{Finance})$ 

In [9]:
x = words.loc[10,:]

def compute_p_x_cat(x):    
    p_x_cat = [1.0, 1.0, 1.0]
    for c_i, category in enumerate(categories):
        mask = job_titles['job_category'] == category
        n = mask.sum()    
        cols = words.columns    
        print("p(x|%s)= " % category, end='')
        for i, c in enumerate(cols[:-1]):
            nx = (job_titles[mask][c] == x[i]).sum()
            print("{}/{} * ".format(nx, n), end='')
            p_x_cat[c_i] *= nx*1.0/n
        i += 1
        c = cols[-1]
        nx = (job_titles[mask][c] == x[i]).sum()
        print("{}/{} = ".format(nx, n), end='')
        p_x_cat[c_i] *= nx*1.0/n
        print("%.5f \n" % p_x_cat[c_i])
    return p_x_cat
        
p_x_cat = compute_p_x_cat(x)

p(x|finance)= 1/3 * 3/3 * 1/3 * 2/3 * 1/3 * 3/3 * 2/3 * 3/3 * 3/3 * 1/3 = 0.00549 

p(x|sales)= 0/4 * 3/4 * 0/4 * 3/4 * 3/4 * 3/4 * 4/4 * 0/4 * 3/4 * 4/4 = 0.00000 

p(x|technology)= 2/3 * 3/3 * 3/3 * 3/3 * 1/3 * 3/3 * 3/3 * 3/3 * 2/3 * 3/3 = 0.14815 



---

###  $p(\mathbf{x})$

We run into the same issue as before when we try to estimate $p(\mathbf{x})$. According to our training data $p(\mathbf{x}) = 0$, but the true value of $p(\mathbf{x})$ should obviously be > 0. However, consider our end-goal which is to determine $p(\text{Technology} \mid \mathbf{x})$, $p(\text{Sales} \mid \mathbf{x})$, and $p(\text{Finance} \mid \mathbf{x})$. Per Bayes’ Theorem, these probabilities are equivalent to

$ p(\text{Technology} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Technology})p(\text{Technology})}{p(\mathbf{x})}$

$ p(\text{Sales} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Sales})p(\text{Sales})}{p(\mathbf{x})} $

$ p(\text{Finance} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Finance})p(\text{Finance})}{p(\mathbf{x})} $

Notice that $p(\mathbf{x})$ is just a scaling factor. It affects the values of $p(\text{Technology} \mid \mathbf{x})$, $p(\text{Sales} \mid \mathbf{x})$, and $p(\text{Finance} \mid \mathbf{x})$ equally, And since these probabilities must sum to 1 we can solve for $p(\mathbf{x})$:

$1 = p(\text{Technology} \mid \mathbf{x}) + p(\text{Sales} \mid \mathbf{x}) + p(\text{Finance} \mid \mathbf{x})$

$\Rightarrow 1 = \frac{p(\text{Technology})p(\mathbf{x} \mid \text{Technology})}{p(\mathbf{x})} + \frac{p(\text{Sales})p(\mathbf{x} \mid \text{Sales})}{p(\mathbf{x})} + \frac{p(\text{Finance})p(\mathbf{x} \mid \text{Finance})}{p(\mathbf{x})}$

$\Rightarrow p(\mathbf{x}) = p(\text{Technology}) p(\mathbf{x} \mid \text{Technology}) + p(\text{Sales}) p(\mathbf{x} \mid \text{Sales}) + p(\text{Finance}) p(\mathbf{x} \mid \text{Finance})$

So, in our example we can calculate $p(\mathbf{x})$ to be

In [10]:
def compute_p_x(p_cat, p_x_cat):
    p_x = 0.0
    for p_c, p_x_c in zip(p_cat, p_x_cat):
        p_x += p_c * p_x_c
    print("p(x)=%.5f" % p_x)
    return p_x

p_x = compute_p_x(p_cat, p_x_cat)

p(x)=0.04609


--- 
### Putting it all together

We can generate probability estimates:

$ p(\text{Technology} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Technology})p(\text{Technology})}{p(\mathbf{x})} $

$ p(\text{Sales} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Sales})p(\text{Sales})}{p(\mathbf{x})} $

$ p(\text{Finance} \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid \text{Finance})p(\text{Finance})}{p(\mathbf{x})}$

In [11]:
def compute_p_cat_x(p_x_cat, p_cat, p_x):
    p_cat_x = [0.0, 0.0, 0.0]
    for i, p_c, c in zip(range(3), p_cat, categories):
        p_cat_x[i] = p_x_cat[i] * p_c / p_x
        print("p(%s|x)=%.5f" % (c, p_cat_x[i]))
    return p_cat_x

p_cat_x = compute_p_cat_x(p_x_cat, p_cat, p_x)

p(finance|x)=0.03571
p(sales|x)=0.00000
p(technology|x)=0.96429


Finally, if we want to turn this model into a classifier we just need to use a decision rule like 

*Label the sample using the class with the highest probability.*

Awesome! But there’s still a nagging issue… Suppose we try to classify the title “junior data analyst”. With our current model, we’d get

In [12]:
x = words.loc[11,:]
p_x_cat = compute_p_x_cat(x)

p(x|finance)= 1/3 * 3/3 * 1/3 * 1/3 * 2/3 * 3/3 * 2/3 * 3/3 * 3/3 * 1/3 = 0.00549 

p(x|sales)= 0/4 * 3/4 * 0/4 * 1/4 * 1/4 * 3/4 * 4/4 * 0/4 * 3/4 * 4/4 = 0.00000 

p(x|technology)= 2/3 * 3/3 * 3/3 * 0/3 * 2/3 * 3/3 * 3/3 * 3/3 * 2/3 * 3/3 = 0.00000 



Our intuition tells us that “junior data analyst” is more likely to be a Technology job than a Finance job, but since the word “junior” never appeared in any of the Technology training samples, $p(x_{4}=1 \mid \text{Technology}) = 0$ which will ultimately cause $p(\text{Technology} \mid \mathbf{x})$ to be 0. In order to deal with this issue, we’ll introduce something called [Additive Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) which will ensure that $p(x_i \mid C_k)$ is never 0 by making

$$ p(x_i \mid C_k) = \frac{\text{frequency}(x_i \mid C_k) + \alpha}{\text{frequency}(C_k) + \alpha n_i},  $$
where $n_i$ is the number of possible values for $x_i$ (in our examples, 2).

If $\alpha=1$ this is known as Laplace Smoothing. In our examples, Laplace Smoothing produces the following: 

In [13]:
def compute_p_x_cat(x, alpha, n_i):    
    p_x_cat = [1.0, 1.0, 1.0]
    for c_i, category in enumerate(categories):
        mask = job_titles['job_category'] == category
        n = mask.sum()    
        cols = words.columns    
        print("p(x|%s)= " % category, end='')
        for i, c in enumerate(cols[:-1]):
            nx = (job_titles[mask][c] == x[i]).sum()
            nx += alpha
            nn = n + alpha * n_i
            print("{}/{} * ".format(nx, nn), end='')
            p_x_cat[c_i] *= nx*1.0/nn
        i += 1
        c = cols[-1]
        nx = (job_titles[mask][c] == x[i]).sum()
        nx += alpha
        nn = n + alpha * n_i
        print("{}/{} = ".format(nx, nn), end='')
        p_x_cat[c_i] *= nx*1.0/nn
        print("%.5f \n" % p_x_cat[c_i])
    return p_x_cat

p_x_cat = compute_p_x_cat(x, alpha=1.0, n_i=2)

p(x|finance)= 2.0/5.0 * 4.0/5.0 * 2.0/5.0 * 2.0/5.0 * 3.0/5.0 * 4.0/5.0 * 3.0/5.0 * 4.0/5.0 * 4.0/5.0 * 2.0/5.0 = 0.00377 

p(x|sales)= 1.0/6.0 * 4.0/6.0 * 1.0/6.0 * 2.0/6.0 * 2.0/6.0 * 4.0/6.0 * 5.0/6.0 * 1.0/6.0 * 4.0/6.0 * 5.0/6.0 = 0.00011 

p(x|technology)= 3.0/5.0 * 4.0/5.0 * 4.0/5.0 * 1.0/5.0 * 3.0/5.0 * 4.0/5.0 * 4.0/5.0 * 4.0/5.0 * 3.0/5.0 * 4.0/5.0 = 0.01132 



In [14]:
p_x = compute_p_x(p_cat, p_x_cat)
p_cat_x = compute_p_cat_x(p_x_cat, p_cat, p_x)

p(x)=0.00457
p(finance|x)=0.24769
p(sales|x)=0.00926
p(technology|x)=0.74306


---
### Generalizing this

Recall our model which is a direct translation of Bayes’ Theorem

$$ p(C_k \mid \mathbf{x}) = \frac{ p(\mathbf{x} \mid C_k) p(C_k) }{p(\mathbf{x})} $$

From our naive assumption, we transformed $p(\mathbf{x} \mid C_k)$ into the product $\prod_{i=1}^n p(x_i \mid C_k)$. Then we calculated $p(x_i \mid C_k)$ using relative frequencies. The underlying argument for using relative frequencies is that $x_i$ was a  [Bernoulli Random Variable](https://en.wikipedia.org/wiki/Bernoulli_distribution) (because $x_i$ is either 0 or 1 for our example). However, in the general case $x_i$ can be from any distribution and our Naive Bayes model will still work as long as we can closely estimate $p(x_i \mid C_k)$. This includes [multinomial random variables](https://en.wikipedia.org/wiki/Multinomial_distribution) (commonly used in document classification problems like ours, but where word-repeats can occur) as well as continuous random variables. That means we can include things like a user’s bio (word frequencies) and age as additional features in our model.